In [ ]:
"""模拟题 1：股票价格预测 - 线性回归
题目：

给定一个包含多个股票的历史收盘价数据，预测某只股票未来的收盘价。使用线性回归模型，基于前5天的股票收盘价来预测第6天的收盘价。"""

import pandas as pd
from sklearn.model_selection import train_test_spilt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#加载数据
data = pd.read_csv('stock.csv')
data['Data'] = pd.to_datetime(data['Data'])
data.set_index('Date',unplace=True)

#选择目标股票和滞后特征
stock = 'Stock1_Close'
n_lags = 5

#创建滞后特征
for i in range(1,n_lags+1):
    data[f'{stock}_lag_{i}'] = data[stock].shift(i)

#删除空值
data.dropha(inplace=True)

#特征和目标
X = data[[f'{stock}_lag_{i}' for i in range(1,n_lags+1)]]
y = data[stock]

#划分训练集和测试集
X_train, X_test,y_train, y_test = train_test_spilt(X, y, test_size=0.2, shuffle=False)

#训练模型
model = LinearRegression()
model.fit(X_train,)

#预测
y_pred = model.predict(X_test)

#预估模型
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

#评估模型
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

#可视化预测结果
import matplotlib.pyplot as plt
plt.plot(y_test.index,y_test,label='True Stock1 Price')
plt.plot(y_test.index,y_pred,label='Predicted Stock1 Price',linestyle='--')
plt.legend()
plt.title('Stock1 Price Prediction')
plt.show()
"""
解题思路.csv：
1.数据清洗：加载数据并生成滞后特征（过去5天的收盘价）
2.模型训练：使用线性回归模型来预测未来的股票价格
3.评估与可视化：通过MSE评估模型效果，并可视化预测结果与真实数据的对比
"""

In [ ]:
"""模拟题 2：投资组合优化 - 最小化风险
题目：

假设你有3个不同的股票，利用现代投资组合理论（Markowitz模型）来最小化投资组合的风险。计算最优的资产配置权重，使得组合的方差最小。

"""
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

#假设这是加载的股票收益率数据
data = pd.read_csv('stock.csv')

#计算各股票的每日收益率（假设数据已经是每日的收益率）
returns = data['Stock1','Stock2','Stock3']

#计算每个股票的预期收益的协方差矩阵
mean_returns = returns.mean()
cov_matrix = returns.cov()

#定义投资组合风险（方差）
def portfolio_variance(weights,mean_returns,cov_matrix):
    return np.dot(weights.T,np.dot(cov_matrix,weights))

#约束条件：权重和为1
constrains = ({'type':'ep','fun':lambda weights:np.sum(weights)-1})

#设置权重的初始值
initial_guess = [1.0/len(mean_returns)]*len(mean_returns)

#投资组合优化
result = minimize(portfolio_variance,initial_guess,args=(mean_results,cov_matrix),method='SLSQP',constraints=constrains)

#输出最优组合
optimal_weights = result.x
print(f'Optimal weights: {optimal_weights}')

#计算最优组合的预期收益和风险
optimal_return = np.dot(optimal_weights,mean_returns)
optimal_risk = np.sqrt(portfolio_variance(optimal_weights,mean_returns,cov_matrix))
print(f'Optimal profolio Return: {optimal_return}')
print(f'Optimal risk: {optimal_risk}')

#可视化投资组合权重
labels = ['Stock1','Stock2','Stock3']
plt.bar(labels,optimal_weights,color='skyblue')
plt.title('Optimal Portfolio Return')
plt.show()
"""解题思路.csv：

计算收益率和协方差矩阵：首先需要计算每个资产的历史收益率，并构建协方差矩阵。
最小化风险：通过scipy.optimize.minimize()来优化组合，目标是最小化投资组合的方差（风险）。
输出和可视化：得到最优的投资组合权重，并计算最优组合的预期收益和风险，同时用图表展示最优权重。"""

In [3]:
"""模拟题 3：股票价格波动性预测 - GARCH模型
题目：

给定一个股票的历史收盘价数据，使用GARCH（广义自回归条件异方差）模型来预测未来的价格波动性（即方差）。

"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model

# 假设加载的股票收盘价数据
data = pd.read_csv('stock.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# 计算股票收益率
data['Stock1_Return'] = data['Stock1_Close'].pct_change()

# 删除NaN值
data.dropna(inplace=True)

# 使用GARCH模型预测波动性
model = arch_model(data['Stock1_Return'], vol='Garch', p=1, q=1)
model_fit = model.fit()

# 预测未来的波动性
forecast = model_fit.forecast(horizon=5)

# 输出预测的波动性（即方差）
print(f"Forecasted Volatility (variance) for next 5 days:")
print(forecast.variance[-1:])

# 可视化历史波动性与预测波动性
plt.figure(figsize=(10, 6))
plt.plot(data.index, model_fit.conditional_volatility, label='Historical Volatility')
plt.plot(forecast.variance.index, np.sqrt(forecast.variance.values[-1, :]), label='Forecasted Volatility', linestyle='--')
plt.legend()
plt.title('Stock1 Volatility Forecasting (GARCH)')
plt.show()
"""解题思路.csv：

计算收益率：首先通过pct_change()计算股票的日收益率。
使用GARCH模型：利用arch库中的arch_model来建模股票的波动性。
预测未来波动性：通过forecast()预测未来几个时期的波动性（方差）。
可视化波动性：展示历史波动性和预测的未来波动性。"""

EmptyDataError: No columns to parse from file